In [2]:
#import statements
import numpy as np
from astropy.io import ascii
from astropy.table import Table, Column
from astropy import units as u

#read in results from MCMC fitting
results=ascii.read('/users/adignan/results_mcmc_filt.csv',format='csv')

#path to MCMC input files
path='/users/adignan/mcmcfiles/'

#initialize empty lists
thermalfrac=[]
thermalfracposunc=[]
thermalfracnegunc=[]

#define equation for calculating free-free emission
def I_ff(nu, T, Aff):
    def gff(nu, T):
        T4 = T/(1.e4*u.K)
        nu9 = nu/(1.*u.GHz)
        gnu = np.log(np.exp(5.960 - (np.sqrt(3.)/np.pi)*np.log(nu9*(T4**(-3./2.)))) + np.exp(1.))
        return gnu   
    
    return Aff*gff(nu,T)/gff(30.*u.GHz,T)

#for loop to calculate thermal fractions and uncertainties
for row in results:
    
    file=row['sourceid']
    
    #read in MCMC file
    (nu, fnu, sfnu) = np.genfromtxt(path+file, unpack=True)

    #calculate thermal fraction based on best-fit result
    tfrac= I_ff(33*u.GHz,1.e4*u.K,row['A_ff median'])/fnu[2]
    thermalfrac.append(tfrac)
    
    #calculate positive uncertainty based on best-fit result uncertainty
    tfracuncpos=np.sqrt(  ( I_ff(33*u.GHz,1.e4*u.K,row['A_ff one sigma + error'])/I_ff(33*u.GHz,1.e4*u.K,row['A_ff median']) )**2 
                     + ( sfnu[2]/fnu[2] )**2 )
    thermalfracposunc.append(tfracuncpos)
    
    #calculate negative uncertainty based on best-fit result uncertainty
    tfracuncneg=np.sqrt(  (I_ff(33*u.GHz,1.e4*u.K,row['A_ff one sigma – error'])/I_ff(33*u.GHz,1.e4*u.K,row['A_ff median']) )**2 
                     + ( sfnu[2]/fnu[2] )**2 )
    thermalfracnegunc.append(tfracuncneg)

    #print statements for looking at results without saving it to a csv file
#     print(r'tfrac: '+"%.3f" % tfrac + ' + ' +"%.3f" % tfracuncpos
#          + ' - ' +"%.3f" % tfracuncneg)

#make astropy table of results
idscol=Column(results['sourceid'],name='source ID')
fraccol=Column(thermalfrac,name='thermal fraction')
poscol=Column(thermalfracposunc,name='positive uncertainty')
negcol=Column(thermalfracnegunc,name='negative uncertainty')

tbl1=Table([idscol,fraccol,poscol,negcol])
#write results to csv file
tbl1.write('/users/adignan/thermalfractions_mcmc.csv',format='csv',overwrite=True)